# **Extract Transform Load (ETL)**

In [1]:
import glob                         # this module helps in selecting files 
import pandas as pd                 # this module helps in processing CSV files
import xml.etree.ElementTree as ET  # this module helps in processing XML files.
from datetime import datetime

## Download Files

In [4]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip

--2022-11-25 17:44:31--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4249 (4.1K) [application/zip]
Saving to: ‘datasource.zip’

datasource.zip      100%[===================>]   4.15K  --.-KB/s    in 0s      

2022-11-25 17:44:31 (30.2 MB/s) - ‘datasource.zip’ saved [4249/4249]



In [5]:
!unzip datasource.zip -d dealership_data

Archive:  datasource.zip
  inflating: dealership_data/used_car_prices1.csv  
  inflating: dealership_data/used_car_prices2.csv  
  inflating: dealership_data/used_car_prices3.csv  
  inflating: dealership_data/used_car_prices1.json  
  inflating: dealership_data/used_car_prices2.json  
  inflating: dealership_data/used_car_prices3.json  
  inflating: dealership_data/used_car_prices1.xml  
  inflating: dealership_data/used_car_prices2.xml  
  inflating: dealership_data/used_car_prices3.xml  


## About the Data

## Set Paths

In [7]:
tmpfile    = "dealership_temp.tmp"               # file used to store all extracted data
logfile    = "dealership_logfile.txt"            # all event logs will be stored in this file
targetfile = "dealership_transformed_data.csv"   # file where transformed data is stored

## Extract

### CSV Extract Function

In [8]:
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

### JSON Extract Function

In [9]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process, lines=True)
    return dataframe

### XML Extract Function

In [25]:
def extract_from_xml(file_to_process):
    dataframe= pd.DataFrame(columns=['car_model', 'year_of_manufacture', 'price', 'fuel'])
    tree= ET.parse(file_to_process)
    root= tree.getroot()
    for person in root:
        car_model= person.find('car_model').text
        year_of_manufacture= int(person.find('year_of_manufacture').text)
        price=  float(person.find('price').text)                        
        fuel= person.find('fuel').text
        dataframe= dataframe.append({'car_model': car_model, 'year_of_manufacture': year_of_manufacture, 'price': price, 'fuel': fuel}, ignore_index =True)
    return dataframe

### Extract Function

In [16]:
def extract():
    extracted_data = pd.DataFrame(columns=['car_model', 'year_of_manufacture', 'price', 'fuel'])  # create an empty data frame to hold extracted data
    
    # process all csv files
    for csvfile in glob.glob('dealership_data/*.csv'):
        extracted_data= extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
                             
    # process all json files
    for jsonfile in glob.glob('dealership_data/*.json'):
        extracted_data= extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    
    # process all xml files
    for xmlfile in glob.glob('dealership_data/*.xml'):
        extracted_data= extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
                             
    return extracted_data    

## Transform 

In [30]:
# transform function
def transform(data):
    # round the price columns to 2 decimal places
    data['price']= round(data.price, 2) 
    return data

## Loading

In [19]:
# load function
def load(targetfile, data_to_load):
    data_to_load.to_csv(targetfile)

## Logging

In [20]:
def log(message):
    timestamp_format = '%H:%M:%S-%h-%d-%Y' #Hour-Minute-Second-MonthName-Day-Year
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("dealership_logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n') 

## Running ETL Process

In [22]:
# Log started the ETL process
log('etl process started')

In [23]:
# Log started the Extract step
log('extract process started')

In [26]:
# Call the Extract function
extracted_data = extract()

In [27]:
# Log completion of  the Extract step
log('extraction completed')

In [28]:
# Log start of Transform step
log('started transform step')

In [32]:
# Call the Transform function
transformed_data= transform(extracted_data)

In [33]:
# Log completion of Transform step
log('transformaton step completed')

In [34]:
# Log start of the Load step
log('start of load step')

In [35]:
# Call the Load function
load(targetfile, transformed_data)

In [36]:
# Log completed the Load step
log("Load phase Ended")

In [37]:
# Log completion of the ETL process
log("ETL Job Ended")